In [42]:

import pandas as pd
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import re
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import RobustScaler
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
df=pd.read_excel('../data/processed/combined_profiles_sorted.xlsx')

In [10]:
df

,age,location,ethnicity,occupation,status,description,scam,age_group,country
0,35,"Kabul, Afghanistan",native american,military,single,I love to laugh and to enjoy the simple things...,1,31-40,Afghanistan
1,41,"Phoenix, Arizona, United States or Kabul, Afgh...",white,military,divorced,"I?€?m a honest, loving, and trust worthy man l...",1,41-50,Afghanistan
2,46,"Charlotte, North Carolina, United States, or K...",native american,military,widowed,My heart is like an open book it?€?s depend on...,1,41-50,Afghanistan
3,49,"Rockville, Maryland, United States, or Kabul, ...",white,government,widowed,"Hello,i am here to look for a good relationshi...",1,41-50,Afghanistan
4,47,"Kabul, Afghanistan",white,military,widowed,"My name is Jeffery Yates, I am 47 years old an...",1,41-50,Afghanistan
...,...,...,...,...,...,...,...,...,...
5964,30,"Ho Chi Minh City, Ho Chi Minh, Vietnam",asian,self-employed,single,"An open-minded, friendly and helpful girl! A t...",0,31-40,Vietnam
5965,28,"Zabid, Yemen",white,military,single,I am single definitely no kid. My personality ...,1,21-30,Yemen
5966,61,"Mpika, Zambia",white,self-employed,separated,"Loves life with plenty of energy, zeal and ent...",0,61-70,Zambia
5967,55,"Lusaka, Zambia",black,artist,separated,"easy going, adventurous, creative. Loves to so...",0,51-60,Zambia


In [11]:
categorical = [var for var in df.columns if df[var].dtype=='O']
for var in categorical:
    
    print(var, ' contains ', len(df[var].unique()), ' labels')

location  contains  2804  labels
ethnicity  contains  18  labels
occupation  contains  39  labels
status  contains  6  labels
description  contains  5837  labels
age_group  contains  8  labels
country  contains  162  labels


In [12]:
y=df['scam']
X=df.drop(columns=['scam','age','location'])

In [13]:
X

,ethnicity,occupation,status,description,age_group,country
0,native american,military,single,I love to laugh and to enjoy the simple things...,31-40,Afghanistan
1,white,military,divorced,"I?€?m a honest, loving, and trust worthy man l...",41-50,Afghanistan
2,native american,military,widowed,My heart is like an open book it?€?s depend on...,41-50,Afghanistan
3,white,government,widowed,"Hello,i am here to look for a good relationshi...",41-50,Afghanistan
4,white,military,widowed,"My name is Jeffery Yates, I am 47 years old an...",41-50,Afghanistan
...,...,...,...,...,...,...
5964,asian,self-employed,single,"An open-minded, friendly and helpful girl! A t...",31-40,Vietnam
5965,white,military,single,I am single definitely no kid. My personality ...,21-30,Yemen
5966,white,self-employed,separated,"Loves life with plenty of energy, zeal and ent...",61-70,Zambia
5967,black,artist,separated,"easy going, adventurous, creative. Loves to so...",51-60,Zambia


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
categorical

['location',
 'ethnicity',
 'occupation',
 'status',
 'description',
 'age_group',
 'country']

In [18]:
X_train

,ethnicity,occupation,status,description,age_group,country
2409,white,technology,separated,"I'm going by right,639 I am a very good friend...",61-70,Spain
1173,white,carer,single,"Am loving,caring,honest,passionate,faithful an...",31-40,Germany
3220,other,engineering,divorced,I?€?m ready to explore the next chapter in my ...,51-60,United States
1669,hispanic,other,single,"40 good vibes, very attentive, respectful, a l...",41-50,Mexico
944,hispanic,technology,single,"I am a normal, happy, fun person. I don't drin...",41-50,Ecuador
...,...,...,...,...,...,...
3772,white,other,divorced,well im a cool person who travels a lot i love...,51-60,United States
5191,hispanic,other,single,"I like them older, I want to have a good time.",21-30,United States
5226,white,other,divorced,I am a good humored single father hard working...,41-50,United States
5390,white,sales,divorced,"Hello, I am dual citizen living in New York an...",51-60,United States


In [19]:
# encode remaining variables with one-hot encoding

encoder = ce.OneHotEncoder(cols=['ethnicity', 'occupation', 'status', 'age_group','country'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

In [37]:
stemmer = PorterStemmer()
 
def stem_words(text):
    # Split the input sentence into words
    word_tokens = text.split()
    # Stem each word individually
    stems = [stemmer.stem(word) for word in word_tokens]
    # Join the stemmed words back into a sentence
    stemmed_sentence = ' '.join(stems)
    return stemmed_sentence
def to_lower(value):
    # Process the value and return the result
    # For example, convert the value to lowercase
    return value.lower()
# Download the list of stopwords if not already downloaded
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    return ' '.join([word for word in words if word.lower() not in stop_words])
 


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/benedicthalim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
df['description']=df['description'].apply(to_lower)

In [38]:
df['description'] = df['description'].apply(remove_stopwords)

In [40]:
df['stem']=df['description'].apply(stem_words)

In [41]:
column_list = df['stem'].tolist()

# pls add here 

In [24]:


scaler = RobustScaler()

X_train_without_description = scaler.fit_transform(X_train)

X_test_without_description = scaler.transform(X_test)

ValueError: could not convert string to float: "I'm going by right,639 I am a very good friend of my friends and from time to time 541 when I kiss with passion..., when I greet I shake my hand, 547 a hug or a kiss is sincerity, falsehoods and lies are better not to listen to....etc"

In [ ]:
# vectorise for the description 

In [20]:

cols = X_train.columns

Index(['ethnicity_1', 'ethnicity_2', 'ethnicity_3', 'ethnicity_4',
       'ethnicity_5', 'ethnicity_6', 'ethnicity_7', 'ethnicity_8',
       'ethnicity_9', 'ethnicity_10',
       ...
       'country_142', 'country_143', 'country_144', 'country_145',
       'country_146', 'country_147', 'country_148', 'country_149',
       'country_150', 'country_151'],
      dtype='object', length=221)


In [142]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [143]:
X_test = pd.DataFrame(X_test, columns=[cols])

In [144]:
# train a Gaussian Naive Bayes classifier on the training set
from sklearn.naive_bayes import GaussianNB


# instantiate the model
gnb = GaussianNB()


# fit the model
gnb.fit(X_train, y_train)

GaussianNB()

In [145]:
y_pred = gnb.predict(X_test)

y_pred

array([0, 1, 0, ..., 1, 0, 1])

In [150]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
print('Model precision score: {0:0.4f}'. format(precision_score(y_test, y_pred)))
print('Model recall score: {0:0.4f}'. format(recall_score(y_test, y_pred)))
print('Model f1 score: {0:0.4f}'. format(f1_score(y_test, y_pred)))


Model accuracy score: 0.7951
Model precision score: 0.8812
Model recall score: 0.7678
Model f1 score: 0.8206


In [151]:
y_pred_train = gnb.predict(X_train)

y_pred_train

array([1, 1, 0, ..., 0, 0, 0])

In [152]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

Training-set accuracy score: 0.8060


,ethnicity_1,ethnicity_2,ethnicity_3,ethnicity_4,ethnicity_5,ethnicity_6,ethnicity_7,ethnicity_8,ethnicity_9,ethnicity_10,...,status_5,status_6,age_group_1,age_group_2,age_group_3,age_group_4,age_group_5,age_group_6,age_group_7,age_group_8
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4425,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4426,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [164]:
Test=pd.concat([X_train, y_train], axis=1)

In [169]:
Test.corr()
#sns.heatmap(Test.corr())

,"(ethnicity_1,)","(ethnicity_2,)","(ethnicity_3,)","(ethnicity_4,)","(ethnicity_5,)","(ethnicity_6,)","(ethnicity_7,)","(ethnicity_8,)","(ethnicity_9,)","(ethnicity_10,)",...,"(status_6,)","(age_group_1,)","(age_group_2,)","(age_group_3,)","(age_group_4,)","(age_group_5,)","(age_group_6,)","(age_group_7,)","(age_group_8,)",scam
"(ethnicity_1,)",1.000000,-0.298544,-0.308100,-0.198589,-0.541706,-0.280973,-0.141876,-0.055464,-0.068997,-0.055464,...,-0.040996,-0.080135,0.114715,-0.018415,0.041100,-0.065272,0.043077,-0.003182,-0.010078,-0.025440
"(ethnicity_2,)",-0.298544,1.000000,-0.074465,-0.047997,-0.130925,-0.067908,-0.034290,-0.013405,-0.016676,-0.013405,...,-0.018094,0.010733,0.036572,-0.005869,-0.024568,-0.026486,-0.019798,-0.008077,-0.015661,0.008597
"(ethnicity_3,)",-0.308100,-0.074465,1.000000,-0.049533,-0.135116,-0.070082,-0.035388,-0.013834,-0.017210,-0.013834,...,0.020576,0.033668,-0.037316,-0.001404,-0.010504,0.020185,-0.012552,-0.008336,-0.001064,-0.002741
"(ethnicity_4,)",-0.198589,-0.047997,-0.049533,1.000000,-0.087090,-0.045172,-0.022810,-0.008917,-0.011093,-0.008917,...,0.007415,0.010813,-0.043516,0.003754,0.015212,0.023682,-0.006434,0.038041,0.012029,0.034918
"(ethnicity_5,)",-0.541706,-0.130925,-0.135116,-0.087090,1.000000,-0.123219,-0.062219,-0.024323,-0.030258,-0.024323,...,0.052684,0.077406,-0.065004,0.051967,-0.010217,-0.061195,-0.029633,0.004431,-0.028416,-0.011919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(age_group_5,)",-0.065272,-0.026486,0.020185,0.023682,-0.061195,0.218011,0.019918,-0.019336,0.008465,-0.005870,...,-0.016102,-0.224377,-0.234757,-0.234487,-0.103329,1.000000,-0.040987,-0.011651,-0.022590,-0.009541
"(age_group_6,)",0.043077,-0.019798,-0.012552,-0.006434,-0.029633,-0.008572,0.020870,-0.005279,-0.006567,-0.005279,...,0.025070,-0.061257,-0.064091,-0.064017,-0.028210,-0.040987,1.000000,-0.003181,-0.006167,-0.010897
"(age_group_7,)",-0.003182,-0.008077,-0.008336,0.038041,0.004431,-0.007602,-0.003838,-0.001501,-0.001867,-0.001501,...,-0.002025,-0.017413,-0.018218,-0.018197,-0.008019,-0.011651,-0.003181,1.000000,-0.001753,0.020094
"(age_group_8,)",-0.010078,-0.015661,-0.001064,0.012029,-0.028416,0.067046,0.023501,-0.002909,-0.003619,-0.002909,...,-0.003927,-0.033762,-0.035324,-0.035283,-0.015548,-0.022590,-0.006167,-0.001753,1.000000,-0.019780


In [182]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
high_correlation = Test.corr()[Test.corr() <-0.9]
print(high_correlation)

                  (ethnicity_1,)  (ethnicity_2,)  (ethnicity_3,)  \
(ethnicity_1,)               NaN             NaN             NaN   
(ethnicity_2,)               NaN             NaN             NaN   
(ethnicity_3,)               NaN             NaN             NaN   
(ethnicity_4,)               NaN             NaN             NaN   
(ethnicity_5,)               NaN             NaN             NaN   
(ethnicity_6,)               NaN             NaN             NaN   
(ethnicity_7,)               NaN             NaN             NaN   
(ethnicity_8,)               NaN             NaN             NaN   
(ethnicity_9,)               NaN             NaN             NaN   
(ethnicity_10,)              NaN             NaN             NaN   
(ethnicity_11,)              NaN             NaN             NaN   
(ethnicity_12,)              NaN             NaN             NaN   
(ethnicity_13,)              NaN             NaN             NaN   
(occupation_1,)              NaN             NaN